# Título   Crear clase ternaria con duckdb
## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Crear clase ternaria usando duckdb

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [ ]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive



## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [ ]:
# Ejemplo


dataset_path = '~/buckets/b1/datasets' 
dataset_file = 'competencia_02_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [ ]:
output_file='competencia_02_duck.csv.gz'
duckdb_file='competencia_02.db'

In [ ]:
## Procesos

### Paquetes necesarios

In [ ]:
!pip install duckdb

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [ ]:
import os
import pandas as pd
import duckdb

In [ ]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
file_path = os.path.join(dataset_path, dataset_file)
# Conectar a una base de datos en memoria o en un archivo
duckdb_path=os.path.join(dataset_path, duckdb_file)

conn = duckdb.connect(duckdb_path) 


In [ ]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes: int, meses: int) -> int:
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm



In [ ]:

# Registrar la función UDF en DuckDB
conn.create_function('sig_mes', sig_mes)

In [ ]:
# Leer el CSV y cargarlo en una tabla de DuckDB
sql=f"DROP TABLE IF EXISTS competencia_02"
conn.execute(sql)
sql =f"CREATE TABLE competencia_02 AS SELECT * FROM read_csv_auto('{file_path}')"
print (sql)
conn.execute(sql)



In [ ]:
# Verificar si la tabla fue creada correctamente
result = conn.execute("SELECT * FROM competencia_02 LIMIT 5").fetchall()
print(result)
# Contar los registros en la tabla
count = conn.execute("SELECT COUNT(*) FROM competencia_02").fetchone()[0]
print (count)



In [ ]:
sql="""
  SELECT c2.*,
       CASE
           WHEN c1.numero_de_cliente IS NOT NULL THEN TRUE
           ELSE FALSE
       END AS existe1
FROM competencia_02 c2
LEFT JOIN competencia_02 c1
ON c2.numero_de_cliente = c1.numero_de_cliente
   AND c1.foto_mes = sig_mes(c2.foto_mes, 1);
   """
print (sql)

resultado=conn.execute(sql).fetchall()
print (resultado)


In [ ]:
# Cerrar la conexión cuando termines
conn.close()

201104

In [ ]:
# leer el dataset
dirbase="/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf"
dirdatasets=dirdataset = os.path.join(dirbase, 'datasets')
file_path = os.path.join(dirdatasets, 'competencia_01_crudo.csv')

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

   numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0          249221109    202101               1            0         0   
1          249221468    202101               1            0         0   
2          249223005    202101               1            0         0   
3          249228180    202101               1            0         0   
4          249232117    202101               1            0         0   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            61                 300        1664.94              17994.40   
1            53                  37        4879.78              18640.95   
2            48                 202        1828.35              37789.01   
3            66                 320         175.87               2600.67   
4            79                 373       16600.69              32270.07   

   mcomisiones  ...  Visa_madelantodolares  Visa_fultimo_cierre  Visa_mpagado  \
0      1605.81  ...    

In [ ]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=mm//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm

  return año


In [ ]:
# para cada cliente y mes, verificar si está de baja.
df["foto_mes"].unique()

array([202101, 202102, 202103, 202104, 202105, 202106])

In [ ]:
df["cli_mes"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].astype(str)
df["cli_mesMas1"]=df["numero_de_cliente"].astype(str)+(df["foto_mes"]+1).astype(str)
df["cli_mesMas2"]=df["numero_de_cliente"].astype(str)+(df["foto_mes"]+2).astype(str)


In [ ]:
df[["cli_mes", "cli_mesMas1", "cli_mesMas2"]].head()

,cli_mes,cli_mesMas1,cli_mesMas2
0,249221109202101,249221109202102,249221109202103
1,249221468202101,249221468202102,249221468202103
2,249223005202101,249223005202102,249223005202103
3,249228180202101,249228180202102,249228180202103
4,249232117202101,249232117202102,249232117202103


In [ ]:
df1=df["cli_mes"]   #sólo las columnas del cliente
# Realiza la unión entre df y df1
resultado = df.merge(df1, left_on='cli_mesMas1', right_on='cli_mes', how='left', indicator=True)
df['existe1'] = resultado['_merge'] == 'both'
resultado = df.merge(df1, left_on='cli_mesMas2', right_on='cli_mes', how='left', indicator=True)
df['existe2'] = resultado['_merge'] == 'both'




In [ ]:
df[["numero_de_cliente", "foto_mes", "existe1", "existe2"]]

,numero_de_cliente,foto_mes,existe1,existe2
0,249221109,202101,True,True
1,249221468,202101,True,True
2,249223005,202101,True,True
3,249228180,202101,True,True
4,249232117,202101,True,True
...,...,...,...,...
981941,1598444941,202106,False,False
981942,1598471047,202106,False,False
981943,1598515897,202106,False,False
981944,1598517059,202106,False,False


In [ ]:
df[ "clase_ternaria"]="CONTINUA"          #default
df.loc[df["existe2"]==False, "clase_ternaria"]="BAJA+2"
df.loc[df["existe1"]==False, "clase_ternaria"]="BAJA+1"
df.loc[df["existe2"]==True, "clase_ternaria"]="CONTINUA"

In [ ]:
df[["numero_de_cliente", "foto_mes", "existe1", "existe2", "clase_ternaria"]]

,numero_de_cliente,foto_mes,existe1,existe2,clase_ternaria
0,249221109,202101,True,True,CONTINUA
1,249221468,202101,True,True,CONTINUA
2,249223005,202101,True,True,CONTINUA
3,249228180,202101,True,True,CONTINUA
4,249232117,202101,True,True,CONTINUA
...,...,...,...,...,...
981941,1598444941,202106,False,False,BAJA+1
981942,1598471047,202106,False,False,BAJA+1
981943,1598515897,202106,False,False,BAJA+1
981944,1598517059,202106,False,False,BAJA+1


In [ ]:
df['clase_ternaria'].unique()

array(['CONTINUA', 'BAJA+2', 'BAJA+1'], dtype=object)

In [ ]:
# Eliminar los campos creados
df = df.drop(columns=['cli_mes', 'cli_mesMas1', 'cli_mesMas2', 'existe1', 'existe2'])


In [ ]:
nuevo_archivo="competencia_01.csv"
ruta_nuevo_archivo = os.path.join(dirdatasets, nuevo_archivo)
# Guarda el DataFrame en el nuevo archivo
df.to_csv(ruta_nuevo_archivo, index=False)
print(f"El archivo ha sido guardado en: {ruta_nuevo_archivo}")


El archivo ha sido guardado en: /content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/datasets/competencia_01.csv
